In [3]:
def sample_coroutine():
    print("Getting start->")
    x = yield
    print('-> coroutine received:', x)

In [4]:
my_co = sample_coroutine()

In [5]:
my_co

<generator object sample_coroutine at 0x00000231A9561888>

In [6]:
next(my_co)

Getting start->


In [7]:
my_co.send(42)

-> coroutine received: 42


StopIteration: 

In [8]:
next(my_co)

StopIteration: 

In [ ]:
# 仅当协程被激活时，处于暂停状态才能调用send方法

In [9]:
my_co2 = sample_coroutine()

In [10]:
my_co2.send(43) # 必须先激活协程

TypeError: can't send non-None value to a just-started generator

In [11]:
my_co2.send(None)# 可以用next也可以用send(None)激活协程

Getting start->


In [12]:
from inspect import getgeneratorstate # 用inspect库查看协程状态

In [13]:
def sample_coroutine2(a):
    print("Getting start.. a=", a)
    b = yield a
    print("->Received: b =",b)
    c = yield a+b
    print("->Receive c =",c)

In [14]:
my_co3 = sample_coroutine2(14)

In [15]:
# 接下来查看协程变化状态
getgeneratorstate(my_co3) #'GEN_CREATED' 表示处于等待执行状态

'GEN_CREATED'

In [16]:
next(my_co3)

Getting start.. a= 14


14

In [17]:
getgeneratorstate(my_co3) # 'GEN_SUSPENDED'表示在yield处停止

'GEN_SUSPENDED'

In [18]:
my_co3.send(28) # 给b赋值28后yield表达式返回a+b的计算结果，然后进行等待为c赋值的数

->Received: b = 28


42

In [19]:
my_co3.send(99)

->Receive c = 99


StopIteration: 

In [20]:
getgeneratorstate(my_co3)# 'GEN_CLOSED'表示协程处于停止

'GEN_CLOSED'

In [21]:
# 使用协程计算移动平均数
def average():
    total = 0.0
    cout = 0
    result = None
    while True:
        number = yield result
        cout += 1
        total += number
        result = total/cout

In [22]:
av1 = average()

In [23]:
next(av1) # 预激协程

In [24]:
av1.send(10)

10.0

In [25]:
av1.send(20)

15.0

In [26]:
getgeneratorstate(av1)

'GEN_SUSPENDED'

In [27]:
# 预激协程的修饰器
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)      # 预激协程
        return gen
    return primer

In [28]:
# 对修饰器进行测试
@coroutine
def average():
    total = 0.0
    cout = 0
    result = None
    while True:
        number = yield result
        cout += 1
        total += number
        result = total/cout
my_co = average()

In [29]:
my_co.send(20)

20.0

In [30]:
# 协程的退出
# 协程抛出异常退出

In [31]:
my_co.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [32]:
getgeneratorstate(my_co)

'GEN_CLOSED'

In [50]:
# generator.throw 和 close的用法
# 示例
class DemoException(Exception):
    "你想处理的异常类型"

In [51]:
@coroutine
def demo_exc_handing():
    print("-> coroutine start ...")
    while True:
        try:
            x = yield
        except DemoException:
            print("Exception handled , Coutining...")
        else:
            print("-> coroutine received {!r}".format(x))

In [52]:
exc_co = demo_exc_handing()

-> coroutine start ...


In [53]:
exc_co.send(20)

-> coroutine received 20


In [54]:
exc_co.close() # 关闭协程

In [55]:
getgeneratorstate(exc_co)

'GEN_CLOSED'

In [56]:
exc_co2 = demo_exc_handing()

-> coroutine start ...


In [57]:
exc_co2.throw(DemoException) # 在协程中抛出Exception

Exception handled , Coutining...


In [58]:
exc_co2.throw(ZeroDivisionError) # 如果传入无法处理的异常，就抛出异常

ZeroDivisionError: 

In [59]:
getgeneratorstate(exc_co2)

'GEN_CLOSED'

In [116]:
# 让协程返回值
from collections import namedtuple
Result = namedtuple("Result", "Cout Average")
@coroutine
def averager():
    total = 0.0
    cout = 0
    average = None
    while True:
        number = yield average
        if number is None:
            break
        cout += 1
        total += number
        average = total/cout
    return Result(cout,average)
my_co = averager()

In [117]:
my_co.send(10)

10.0

In [118]:
my_co.send(20)

15.0

In [119]:
my_co.send(None) # 观察结果，发现return表达式还是把值赋值给异常StopIteration的一个属性

StopIteration: Result(Cout=2, Average=15.0)

In [121]:
my_co = averager()
my_co.send(10)

10.0

In [122]:
my_co.send(20)

15.0

In [125]:
try:
    my_co.send(None)
except StopIteration as exc:
    print(exc.value)

Result(Cout=2, Average=15.0)


In [126]:
# yield from 用法

In [127]:
def gen():
    yield from 'AB' # for i in 'AB': yield i
    yield from '123'

In [128]:
gen1 = gen()

In [129]:
list(gen1)

['A', 'B', '1', '2', '3']

In [143]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')

# 子生成器
def averager(): 
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield 
        if term is None: 
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average) 

# 委派生成器
def grouper(results, key): 
    while True: 
        results[key] = yield from averager() 
        
# 客户端代码，即调用方
def main(data): 
    results = {}
    for key, values in data.items():
        group = grouper(results, key) 
        next(group) 
        for value in values:
            group.send(value) 
        group.send(None) # 重要！ 
        print("group",group)
        print(results) # 如果要调试，去掉注释
    report(results)
    
# 输出报告
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(result.count, group, result.average, unit))
        
data = {
'girls;kg':
[40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
'girls;m':
[1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
'boys;kg':
[39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
'boys;m':
[1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}
if __name__ == '__main__':
    main(data)


group <generator object grouper at 0x00000231A95CB258>
{'girls;kg': Result(count=10, average=42.040000000000006)}
group <generator object grouper at 0x00000231A9627D58>
{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997)}
group <generator object grouper at 0x00000231A95CB150>
{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224)}
group <generator object grouper at 0x00000231A95CB258>
{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


In [141]:
# result = yield from EXPR的解析
# 即上面代码的result[key] = yield from averager
# 简化版
_i = iter(EXPR) # _i : 子生成器
try:
    _y = next(_i) # 预激后的生成器
except StopIteration as _exc:
    _r = _exc.value 
else：
    while True:
        _s = yield _y 
        try:
            _y = _i.send(_s) # 将调用方的值传入子生成器，
        except StopIteration as _e:
            _r = _e.value
            break
Result = _r

SyntaxError: unexpected EOF while parsing (<ipython-input-141-7d594935e3c6>, line 4)

In [ ]:
# 完整版
_i = iter(EXPR)  # <1>
try:
    _y = next(_i)  # <2>
except StopIteration as _e:
    _r = _e.value  # <3>
else:
    while 1:  # <4>
        try:
            _s = yield _y  # <5>
        except GeneratorExit as _e:  # <6> 
            try:
                _m = _i.close
            except AttributeError:
                pass
            else:
                _m()
            raise _e
        except BaseException as _e:  # <7> 
            _x = sys.exc_info()
            try:
                _m = _i.throw
            except AttributeError:
                raise _e
            else:  # <8>
                try:
                    _y = _m(*_x)
                except StopIteration as _e:
                    _r = _e.value
                    break
        else:  # <9>
            try:  # <10>
                if _s is None:  # <11>
                    _y = next(_i)
                else:
                    _y = _i.send(_s)
            except StopIteration as _e:  # <12>
                _r = _e.value
                break

RESULT = _r  # <13>